In [ ]:
import torch
import requests
from PIL import Image
from io import BytesIO
from diffusers import StableDiffusionImg2ImgPipeline

device = "cuda"
pipe = StableDiffusionImg2ImgPipeline.from_pretrained("nitrosocke/Ghibli-Diffusion", torch_dtype=torch.float32)

In [ ]:
import torch
import requests
from PIL import Image
from io import BytesIO
from diffusers import StableDiffusionImg2ImgPipeline


import torch
from PIL import Image
import requests
from io import BytesIO
import numpy as np

pipe = StableDiffusionImg2ImgPipeline.from_pretrained("nitrosocke/Ghibli-Diffusion", torch_dtype=torch.float32)

# Resmi indirme
url = "https://raw.githubusercontent.com/CompVis/stable-diffusion/main/assets/stable-samples/img2img/sketch-mountains-input.jpg"
response = requests.get(url)
init_image = Image.open(BytesIO(response.content)).convert("RGB")
init_image.thumbnail((768, 768))

# Resmi bir PyTorch tensorine dönüştürme
init_tensor = torch.tensor(np.array(init_image)).permute(2, 0, 1).float()  # RGB kanalları, yükseklik ve genişlik boyutlarını yeniden düzenler

# Eğer gerekiyorsa, tensori normalleştirme (0 ile 1 arasında değerler)
init_tensor /= 255.0

#model = model.float()  # modeli float32 veri türüne dönüştürün (eğer model adı 'model' ise)
init_tensor = init_tensor.float()
